<a href="https://colab.research.google.com/github/icoqueiro/ProcessoSeletivo_AlbertEinstein/blob/main/Q2_TesteVagaDev_AE_IvanCoqueiro_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Questão 2. Data Analysis (15 pontos)



Você tem um arquivo CSV contendo informações de vendas com as colunas: data, produto, quantidade, preco_unitario.

*   Escreva um código que leia o arquivo e calcule o faturamento total por produto.
*   Encontre o produto com maior e menor faturamento.

##Solução

Para trabalhar com dados em PT-BR, fiz a conversão do locale.Após a configuração será necessário resetar o runtime. Selecione 381, depois 3.

In [16]:
!sudo dpkg-reconfigure locales

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
Configuring locales
-------------------

Locales are a framework to switch between multiple languages and allow users to
use their language, country, characters, collation order, etc.

Please choose which locales to generate. UTF-8 locales should be chosen by
default, particularly for new installations. Other character sets may be useful
for backwards compatibility with older systems and software.

  1. All locales                      252. gl_ES ISO-8859-1
  2. C.UTF-8 UTF-8                    253. gl_ES.UTF-8 UTF-8
  3. aa_DJ ISO-8859-1                 254. gl_ES@euro ISO-8859-15
  4. aa_DJ.UTF-8 U

Aqui segue com as alterações dos locales.

In [1]:
import os

# Instalar pt-BR.UTF8
!/usr/share/locales/install-language-pack pt_BR.UTF-8
!dpkg-reconfigure locales

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Generating locales (this might take a while)...
Generation complete.
dpkg-trigger: error: must be called from a maintainer script (or with a --by-package option)

Type dpkg-trigger --help for help about this utility.
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = "en_US",
	LC_ALL = "en_US.UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").
Generating locales (this might take a while)...
  pt_BR.UTF-8... done
Generation complete.


A princípio, iniciaremos as bibliotecas necessárias para realizar a análise dos dados. Também será montado o drive no qual a imagem da base de dados do problema está localizada. Vale ressaltar que essa base foi indicada no problema, mas foi construída com dados aleatórios na plataforma Google Sheets.

In [2]:
#importa as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#monta a imagem do google drive
from google.colab import drive
drive.mount('/content/drive')

#importa os locales
import locale
locale.setlocale(locale.LC_ALL,'pt_BR.UTF-8')
locale.getlocale()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('pt_BR', 'UTF-8')

Agora, os dados serão importados para nossa rotina através da biblioteca pandas. será usado um dataframe para construir a tabela:



In [3]:
dados = pd.read_csv('/content/drive/MyDrive/Processos Seletivos/informacoes_de_vendas.csv')
dados #como é uma tabela curta, plotei os dados para verificar a integridade dos mesmos

,data,produto,quantidade,preco_unitario
0,09/03/2000,mascara de oxigenio,5,"5,98"
1,31/03/2008,tesoura cirurgica,9,"26,32"
2,09/07/2008,esparadrapos,9,"68,17"
3,31/03/1908,agulha hipodermica,10,"26,39"
4,21/12/2015,medicamento a,7,"33,58"
5,13/07/2003,medicamento b,5,"67,94"
6,09/07/2008,medicamento c,9,"41,81"
7,30/11/1999,luva cirurgica,6,"4,25"
8,14/09/1995,hastes flexiveis,6,"44,74"
9,31/03/2008,sabonete,8,"89,10"


In [4]:
dados.dtypes #checa os data types na tabela.

,0
data,object
produto,object
quantidade,int64
preco_unitario,object


In [5]:
#converte as vírgulas em pontos para que seja possível realizar aritméticas
dados['preco_unitario'] = dados['preco_unitario'].apply(lambda v: locale.atof(v.split()[-1]))
dados

,data,produto,quantidade,preco_unitario
0,09/03/2000,mascara de oxigenio,5,5.98
1,31/03/2008,tesoura cirurgica,9,26.32
2,09/07/2008,esparadrapos,9,68.17
3,31/03/1908,agulha hipodermica,10,26.39
4,21/12/2015,medicamento a,7,33.58
5,13/07/2003,medicamento b,5,67.94
6,09/07/2008,medicamento c,9,41.81
7,30/11/1999,luva cirurgica,6,4.25
8,14/09/1995,hastes flexiveis,6,44.74
9,31/03/2008,sabonete,8,89.10


Agora podemos criar uma coluna com os valores faturados. O código está redigido em um número menor de linhas. Nesse caso, pedimos ao dataframe para inserir uma coluna e multiplicamos dentro da própria função a fim de evitar o uso de espaço desnecessário da memória, e otimizar o _runtime_.

In [6]:
dados.insert(4,'faturamento_total',dados['quantidade'].astype(float)*dados['preco_unitario'],allow_duplicates=False)
dados

,data,produto,quantidade,preco_unitario,faturamento_total
0,09/03/2000,mascara de oxigenio,5,5.98,29.90
1,31/03/2008,tesoura cirurgica,9,26.32,236.88
2,09/07/2008,esparadrapos,9,68.17,613.53
3,31/03/1908,agulha hipodermica,10,26.39,263.90
4,21/12/2015,medicamento a,7,33.58,235.06
5,13/07/2003,medicamento b,5,67.94,339.70
6,09/07/2008,medicamento c,9,41.81,376.29
7,30/11/1999,luva cirurgica,6,4.25,25.50
8,14/09/1995,hastes flexiveis,6,44.74,268.44
9,31/03/2008,sabonete,8,89.10,712.80


Produtos sem estoque não faturaram, pois sequer foram vendidos. Segue uma nova tabela sem esses valores

In [7]:
#encontra os produtos com quantidades zeradas de produto
valorzerado = dados.loc[(dados==0).any(axis=1)]
valorzerado
#cria um df sem esses dados zerados
dados_novo = dados.drop(valorzerado.index)
dados_novo

,data,produto,quantidade,preco_unitario,faturamento_total
0,09/03/2000,mascara de oxigenio,5,5.98,29.90
1,31/03/2008,tesoura cirurgica,9,26.32,236.88
2,09/07/2008,esparadrapos,9,68.17,613.53
3,31/03/1908,agulha hipodermica,10,26.39,263.90
4,21/12/2015,medicamento a,7,33.58,235.06
5,13/07/2003,medicamento b,5,67.94,339.70
6,09/07/2008,medicamento c,9,41.81,376.29
7,30/11/1999,luva cirurgica,6,4.25,25.50
8,14/09/1995,hastes flexiveis,6,44.74,268.44
9,31/03/2008,sabonete,8,89.10,712.80


Por fim, definiremos quais são os produtos que mais faturaram e os que menos faturaram. Segue a rotina abaixo com os valores maximos e minimos da tabela:

In [8]:
max_fat = dados_novo.loc[dados_novo['faturamento_total'].idxmax()] #escolhe o produto que possuiu o maior faturamento
min_fat = dados_novo.loc[dados_novo['faturamento_total'].idxmin()] #escolhe o produto que possuiu o menor faturamento

#imprime na tela
str_max = 'maior faturamento foi do item ' + max_fat.produto + ' no valor de ' + 'R${:,.2f}'.format(max_fat.faturamento_total)
str_min = 'menor faturamento foi do item ' + min_fat.produto + ' no valor de ' + 'R${:,.2f}'.format(min_fat.faturamento_total)


#aqui fiz a correção dos sinais de vírgula e de ponto para o formato brasileiro. Aparentemente aqui no colab isso não existe.
str_max = str_max.replace(',',';')
str_max = str_max.replace('.',',')
str_max = str_max.replace(';','.')
str_min = str_min.replace(',',';')
str_min =str_min.replace('.',',')
str_min = str_min.replace(';','.')

print(str_max)
print(str_min)


maior faturamento foi do item ressonancia 1,5T no valor de R$4.502.339,10
menor faturamento foi do item vacina D no valor de R$6,86
